## Treinamento do modelo de `Who X also X`
Treinamento do modelo  de estudo paseado na interação de visualização de produtos num e-commerce, assim  criando um modelo de recomendação `quem-viu-tambem-viu`

### Importação da bibliotecas

In [0]:
import pandas as pd
import numpy as np

### Lendo DataSets de produtos e interações

In [2]:
df_products = pd.read_csv('products.csv')

df_interactions = pd.read_csv('interactions.csv')
df_interactions.head()

,Unnamed: 0,customer_id,product_id,action_id,timestamp
0,0,53c9bdb7-b343-415d-ba56-25603ff76dcd,E9a29F5a,1,1.589035e+09
1,1,c3c5505a-c57b-4a1f-838f-86d3e249829c,d0ffAB6B,1,1.589013e+09
2,2,9f023949-6dd3-42cc-a8ee-1b932e0b3926,f36d28DD,1,1.589033e+09
3,3,389141c7-9555-44c4-b674-b3213f690c8e,c6D43182,1,1.588971e+09
4,4,7664da1d-935f-4ad9-8019-239cdc69ae61,d86fB7dB,1,1.588985e+09


### Gerando Matriz de Usuarios x Produtos vistos

In [3]:
df_interactions = df_interactions[['customer_id', 'product_id', 'action_id']]
users_items_matrix_df = df_interactions.pivot_table(
    index='customer_id',
    columns='product_id',
    values='action_id',
).fillna(0)
users_items_matrix_df.head()

product_id,00D41aFC,02cC9451,02eCbBdd,03eFDBdF,07cd1912,09FE0e2d,0AA09eb1,0AFADcD8,0B5Fda9e,0B92EE66,0Bba6eFC,0C4Ce5fE,0CEbc7ed,0FFf3c25,0aCcE6b3,0bAe8FaA,0cA5C612,0d1db6c7,0d86Ca6E,0eE0C66e,0eEE23DD,0fBD4C79,10320b6B,10da0E5b,1152deF0,11A3eaaE,1207F75e,123Dca63,12f4b8B5,14cB40AE,162cf1F2,184D5Fdc,19ff78B2,1C0D552d,1CBaEF4e,1Cb2A5B4,1bd8A55F,1db079a5,1eF6A9b9,1f1099D6,...,dFe5DeFC,dbcE046B,dd594b9e,ddFabd1A,ddaCCe19,e266e4FA,e543Fa3f,e7EF952d,e9d1b9cf,eACbaaf8,eB3fEf7e,eCFe9EbF,eDBce96F,eECF516c,eFaDA67c,eaDdd61f,eb4CdA9a,ec8AcdEA,ecDCb5F4,ed619bf4,eddfFb7D,ef0B6Cec,efDcBfFA,f2D9bDcE,f2Da3A7f,f3313630,f36d28DD,f4bCF3a3,f5F3baB5,f65E1EfA,f76aF11D,f7C288EC,f86Ca7DC,f91326DE,f9cfA3a0,fFBC6e2A,faF63aa2,fd186AC9,fd2E4977,ffE17DD8
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00007dd7-449d-410f-b506-5c69ccfbe796,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0000aae2-a99a-4595-8e6c-11e676cd703a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000240a8-fd6f-4af3-8fea-0166f0e221f9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0003debf-26a6-47da-9b43-5cf3ecec3807,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00041a2f-07c8-40d3-8575-4b7b0d09f7d8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Modelo Keras com encoder e decoder

Implementão do modelo conténdo um layer no encoder e um no decoder de tamanho 512, o espaço-latente no centro do modelo tem 256 dimensões.

In [4]:
from keras.optimizers import Adam
from keras.layers import Input, Dense, Dropout
from keras.models import Model

def autoEncoder(X):
  '''
  Autoencoder for Collaborative Filter Model
  '''

  # Input
  input_layer = Input(shape=(X.shape[1],), name='UserScore')

  # Encoder
  # -----------------------------
  enc = Dense(512, activation='selu', name='EncLayer1')(input_layer)

  # Latent Space
  # -----------------------------
  lat_space = Dense(256, activation='selu', name='LatentSpace')(enc)
  lat_space = Dropout(0.8, name='Dropout')(lat_space) # Dropout

  # Decoder
  # -----------------------------
  dec = Dense(512, activation='selu', name='DecLayer1')(lat_space)

  # Output
  output_layer = Dense(X.shape[1], activation='linear', name='UserScorePred')(dec)

  # this model maps an input to its reconstruction
  model = Model(input_layer, output_layer)    

  return model

Using TensorFlow backend.


### Instanciando o modelo

In [5]:
# input
X = users_items_matrix_df.values
y = users_items_matrix_df.values

# Build model
model = autoEncoder(X)
model.compile(optimizer = Adam(lr=0.0001), loss='mse')

model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
UserScore (InputLayer)       (None, 500)               0         
_________________________________________________________________
EncLayer1 (Dense)            (None, 512)               256512    
_________________________________________________________________
LatentSpace (Dense)          (None, 256)               131328    
_________________________________________________________________
Dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
DecLayer1 (Dense)            (None, 512)               131584    
_________________________________________________________________
UserScorePred (Dense)        (None, 500)               256500    
Total params: 775,924
Trainable params: 775,924
Non-trainable params: 0
_____________________________________________________

### Treinando o modelo

In [6]:
# Train
hist = model.fit(x=X, y=y,
                  epochs=50,
                  batch_size=64,
                  shuffle=True,
                  validation_split=0.1)

Train on 63000 samples, validate on 7000 samples
Epoch 1/50
63000/63000 [==============================] - 20s 315us/step - loss: 0.0109 - val_loss: 0.0020
Epoch 2/50
63000/63000 [==============================] - 20s 311us/step - loss: 0.0023 - val_loss: 0.0018
Epoch 3/50
63000/63000 [==============================] - 20s 312us/step - loss: 0.0020 - val_loss: 0.0017
Epoch 4/50
63000/63000 [==============================] - 20s 310us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 5/50
63000/63000 [==============================] - 20s 311us/step - loss: 0.0019 - val_loss: 0.0017
Epoch 6/50
63000/63000 [==============================] - 19s 307us/step - loss: 0.0018 - val_loss: 0.0017
Epoch 7/50
63000/63000 [==============================] - 19s 305us/step - loss: 0.0018 - val_loss: 0.0016
Epoch 8/50
63000/63000 [==============================] - 19s 308us/step - loss: 0.0018 - val_loss: 0.0016
Epoch 9/50
63000/63000 [==============================] - 19s 304us/step - loss: 0.0018 - val_l

### Recomendação de Conteúdo para o Usuário

In [7]:
# Predict new Matrix Interactions, set score zero on visualized games
new_matrix = model.predict(X) * (X == 0)

# converting the reconstructed matrix back to a Pandas dataframe
new_users_items_matrix_df  = pd.DataFrame(new_matrix, 
                                          columns = users_items_matrix_df.columns, 
                                          index   = users_items_matrix_df.index)
new_users_items_matrix_df.head()

product_id,00D41aFC,02cC9451,02eCbBdd,03eFDBdF,07cd1912,09FE0e2d,0AA09eb1,0AFADcD8,0B5Fda9e,0B92EE66,0Bba6eFC,0C4Ce5fE,0CEbc7ed,0FFf3c25,0aCcE6b3,0bAe8FaA,0cA5C612,0d1db6c7,0d86Ca6E,0eE0C66e,0eEE23DD,0fBD4C79,10320b6B,10da0E5b,1152deF0,11A3eaaE,1207F75e,123Dca63,12f4b8B5,14cB40AE,162cf1F2,184D5Fdc,19ff78B2,1C0D552d,1CBaEF4e,1Cb2A5B4,1bd8A55F,1db079a5,1eF6A9b9,1f1099D6,...,dFe5DeFC,dbcE046B,dd594b9e,ddFabd1A,ddaCCe19,e266e4FA,e543Fa3f,e7EF952d,e9d1b9cf,eACbaaf8,eB3fEf7e,eCFe9EbF,eDBce96F,eECF516c,eFaDA67c,eaDdd61f,eb4CdA9a,ec8AcdEA,ecDCb5F4,ed619bf4,eddfFb7D,ef0B6Cec,efDcBfFA,f2D9bDcE,f2Da3A7f,f3313630,f36d28DD,f4bCF3a3,f5F3baB5,f65E1EfA,f76aF11D,f7C288EC,f86Ca7DC,f91326DE,f9cfA3a0,fFBC6e2A,faF63aa2,fd186AC9,fd2E4977,ffE17DD8
customer_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00007dd7-449d-410f-b506-5c69ccfbe796,0.000439,-0.003134,-0.003933,0.005740,0.001649,-0.008280,-0.004824,0.001822,-0.001507,0.024398,-0.009083,-0.002934,-0.001013,-0.000872,0.023602,0.013121,-0.005080,0.025775,-0.004919,0.000496,-0.006696,-0.001414,-0.008545,0.012700,-0.000871,-0.001941,-0.005554,-0.004121,0.023006,0.001032,0.004408,0.013980,0.005866,-0.002928,0.002019,-0.004459,-0.011511,-0.000818,-0.003016,0.008313,...,-0.013924,0.003495,0.003270,-0.007419,-0.013300,-0.001487,-0.003690,0.001980,-0.006768,-0.007009,0.009667,0.003901,0.003475,-0.006388,0.000365,-0.000422,0.009140,0.010619,-0.000082,-0.010476,0.013733,0.002393,0.002610,0.008474,0.011599,-0.010033,0.019491,-0.001107,-0.005180,-0.015085,-0.010416,-0.000721,-0.006005,0.000580,0.016713,-0.002800,-0.004007,-0.002240,0.020915,-0.005795
0000aae2-a99a-4595-8e6c-11e676cd703a,0.009510,0.008811,0.008630,0.004102,0.002097,0.001217,-0.002981,0.002823,0.003483,0.000213,-0.009220,0.017588,0.011194,0.001847,0.013389,-0.001672,0.010092,-0.002184,-0.006017,0.003819,0.009679,-0.000773,-0.001704,0.026272,0.003557,0.000703,-0.007457,0.013405,-0.008121,-0.000651,-0.009524,0.011066,-0.006028,0.000376,-0.004175,-0.000280,-0.018072,0.005353,0.001106,0.009065,...,-0.002924,0.000297,-0.003432,-0.011408,-0.008697,0.000675,-0.004807,0.020923,-0.005302,0.025616,0.006129,0.000757,-0.001774,-0.009252,0.010113,-0.001113,-0.006279,0.020892,0.003090,0.004538,0.000269,-0.012987,-0.002888,0.008268,0.000408,-0.002975,0.006922,0.026500,0.007930,0.014762,0.002123,0.000866,0.000347,0.001419,0.009006,-0.011938,0.020072,-0.008266,-0.000695,-0.010723
000240a8-fd6f-4af3-8fea-0166f0e221f9,0.064118,-0.005938,0.004003,0.004569,-0.003805,-0.000057,-0.014925,0.002731,0.002019,-0.004950,0.001943,0.014521,-0.006767,0.010201,-0.002257,-0.003771,-0.006264,0.010583,0.002360,0.029730,0.003455,-0.010639,0.004745,0.007387,-0.009600,0.004237,0.001131,-0.011709,0.007188,0.006829,-0.006268,0.003057,0.006198,-0.007974,-0.004154,0.006682,-0.006406,0.006208,-0.005209,0.015116,...,-0.005192,-0.005927,-0.004449,-0.002421,-0.010287,-0.005027,0.000390,0.005683,0.006573,-0.011602,0.007651,0.012597,-0.007854,-0.010445,-0.002678,0.000932,-0.005232,0.001821,-0.011610,0.008634,0.003616,-0.005924,-0.000242,-0.000104,-0.012193,0.001433,0.020040,-0.004479,0.002694,0.020469,0.010751,-0.004515,0.018012,0.000585,0.007793,-0.008249,0.006850,0.009118,-0.004432,-0.002452
0003debf-26a6-47da-9b43-5cf3ecec3807,-0.000192,0.001464,-0.000255,0.001251,-0.016879,-0.004669,0.006927,0.011404,0.010311,0.002661,0.006494,-0.003587,0.007602,-0.002163,0.025907,0.005138,0.007242,0.012765,-0.001202,0.008513,0.017453,0.017526,0.012209,0.020273,0.002553,0.009898,0.002535,-0.009775,-0.009244,0.000565,0.013620,0.010410,-0.011081,-0.008638,0.004464,0.023892,-0.017449,0.014632,0.011132,0.004741,...,-0.011132,0.017002,0.001902,-0.003973,-0.007380,0.027831,0.002107,0.002502,-0.012659,-0.009416,0.009492,-0.002696,0.012604,0.048178,-0.007869,-0.007571,0.006156,0.006976,0.016163,-0.006372,0.000576,0.014225,-0.004945,-0.006609,-0.008482,-0.002753,0.013796,0.019020,-0.006074,-0.008564,-0.000885,-0.004707,0.000721,0.002487,0.012781,-0.004001,0.012826,0.001018,0

In [0]:
def recommender_for_user(user_id, interact_matrix, df_content, topn = 10):
    '''
    Recommender Games for UserWarning
    '''
    pred_scores = interact_matrix.loc[user_id].values

    df_scores   = pd.DataFrame({'product_id': list(users_items_matrix_df.columns), 
                               'score': pred_scores})

    df_rec      = df_scores.set_index('product_id')\
                    .join(df_content.set_index('sku'))\
                    .sort_values('score', ascending=False)\
                    .head(topn)[['score']]
    
    return df_rec[df_rec.score > 0]

In [13]:
# Recommended User Games
recommender_for_user(user_id         = '0000aae2-a99a-4595-8e6c-11e676cd703a', 
                     interact_matrix = new_users_items_matrix_df, 
                     df_content      = df_products)

,score
product_id,
ACbF1e8C,0.048220
E9a29F5a,0.037558
8a9721e9,0.034617
aB1d8F73,0.034177
Fd6766c9,0.032984
9992C52b,0.031964
21cC3B1f,0.031558
4AC083B0,0.029213
4a664df6,0.026790
